In [1]:
from scipy import stats
import numpy as np
from math import log
import math
import numbers
import csv
import ast

# Information Theory

In this section, you will implement the equations necessary to pick the best attribute based on information gain.

In [2]:
def entropy(class_y):
    # Input:            
    #   class_y         : list of class labels (0's and 1's)
    
    # TODO: Compute the entropy (in log base 2) for a list of classes
    
    """
    Example:
       entropy([1,1,0,1,0,0,1,0,1]) = 0.92
    """
    num_points = len(class_y)
    if num_points == 0:
        return 0
    pos_vals = sum(class_y)
    neg_vals = num_points - pos_vals
    
    pos_prod = 0 if pos_vals == 0 else (float(pos_vals) / num_points) * (math.log((float(pos_vals) / num_points), 2))
    neg_prod = 0 if neg_vals == 0 else (float(neg_vals) / num_points) * (math.log((float(neg_vals) / num_points), 2))                                      
    
    return -(pos_prod + neg_prod)
print("hello world")

hello world


In [17]:
def information_gain(y_before_split, y_after_split):
    # Inputs:
    #   y_before_split: the y labels before splitting (0's and 1's)
    #   y_after_split:  the y labels after splitting in two. This is a list of lists.
    #
    # TODO: Compute the information gain given this split.
    
    """
    Example:
    
    previous_y = [0,0,0,1,1,1]
    current_y = [[0,0], [1,1,1,0]]
    
    info_gain = 0.45915
    """
    H_parent = entropy(y_before_split)
    H_rightChild = entropy(y_after_split[1])
    H_leftChild = entropy(y_after_split[0])
    
    num_points = len(y_before_split)
    r_weight = len(y_after_split[1]) / float(num_points)
    l_weight = len(y_after_split[0]) / float(num_points)
    
    gain = H_parent - (r_weight * H_rightChild + l_weight * H_leftChild)
    return gain

previous_y = [0,0,1,1,1,1]
current_y = [[0,1], [1,1,1,0]]
    
information_gain(previous_y,current_y)

0.044110417748401076

# Utilities

This section implements a utility function that splits data (X,y) based on an attribute and its value. This will be used in the learning procedure of your decision tree when "filtering" data through a decision node.

In [18]:
def partition_classes(X, y, split_attribute_idx, split_val):
    # Inputs:
    #   X                   : data containing all attributes
    #   y                   : labels
    #   split_attribute_idx : column index (in X) of the attribute to split on
    #   split_val           : categorical value to divide the split_attribute
    
    # TODO: Partition the data(X) and labels(y) based on the split value - BINARY SPLIT.
    #.      
    #
    # You can perform the partition in the following way
    #   Split the data X into two lists(X_left and X_right) where the first list has all 
    #   the rows where the split attribute is equal to the split value, and the second list
    #   has all the rows where the split attribute is not equal to the split value.
    #   Also create two lists(y_left and y_right) with the corresponding y labels.

    '''
    Example:
    
    X = [[3, 'aa', 0],                 y = [1,
         [1, 'bb', 3],                      1,
         [2, 'cc', 2],                      0,
         [5, 'bb', 0],                      0,
         [4, 'cc', 3]]                      1]
    
    Here, columns 0 and 2 represent numeric attributes, while column 1 is a categorical attribute.

    Consider the case where we call the function with split_attribute = 1 and split_val = 'bb'
    Then we divide X into two lists, one where column 1 is 'bb', and the other where it is not 'bb'.
        
    X_left = [[1, 'bb', 3],                 y_left = [1,
              [5, 'bb', 0]]                           0]
              
    X_right = [[3, 'aa', 0],                y_right = [1,
               [2, 'cc', 2],                           0,
               [4, 'cc', 3]]                           1]
               
    ''' 
    X_orig = np.array(X)
    y_orig = np.array(y)

    if len(np.shape(X_orig)) == 1:
        if X[split_attribute_idx] == split_val:
            X_left = X_orig
            y_left = y_orig
            X_right = np.array()
            y_right = np.array()
        else:
            X_right = X_orig
            y_right = y_orig
            X_left = np.array()
            y_left = np.array()
        return (X_left, y_left, X_right, y_right)
    else:
        split_column = X_orig[:, split_attribute_idx] #returns a row vector
        mask_equals = (split_column == split_val) 
        mask_notEquals = (split_column != split_val)

        
        X_left = X_orig[mask_equals, :]
        X_right = X_orig[mask_notEquals, :]
    
    
        y_left = y_orig[mask_equals]
        y_right = y_orig[mask_notEquals]
    
        X_left = X_left.tolist()
        X_right = X_right.tolist()
        y_left = y_left.tolist()
        y_right = y_right.tolist()
    
        return (X_left, y_left, X_right, y_right)

X = [[3, 'aa', 0],                 
     [1, 'bb', 3],                  
     [2, 'cc', 2],                      
     [5, 'bb', 0],                      
     [4, 'cc', 3]]                  
y = [1,1,0,0,1]
partition_classes(X,y,1,"cc")

([['2', 'cc', '2'], ['4', 'cc', '3']],
 [0, 1],
 [['3', 'aa', '0'], ['1', 'bb', '3'], ['5', 'bb', '0']],
 [1, 1, 0])

# Decision Tree

This section implements a decision tree class that can be used for learning and predicting. 

In [10]:
class DecisionTree(object):
    def __init__(self, minSplitPercent=5, maxDepth=100):
        # Inputs:
        #   minSplitPercent : minimum percent required for a node to be split 
        #   maxDepth        : maximum allowed depth of a tree.
        
        self.tree = {'depth' : 0}                   # create root node, and set depth parameter to 0
        self.minSplitPercent = minSplitPercent;     
        self.maxDepth = maxDepth;
    
    
    def shouldStopSplitting(self, y, depth):
        # Inputs
        #   y     : a list of the y labels (0's and 1's)
        #   depth : the depth of the current node
        #
        # This method returns True if the maximum depth has been reached, or if 
        # the percentage of 0's or 1's in y is below self.minSplitPercent (see __init__)
        # Otherwise, return False.
        #
        # TODO: Your code here
        a = depth >= self.maxDepth
        if len(y) == 0:
            return True
        
        num_labels = len(y)
        num_pos = sum(y)
        p_y1 = (num_pos / float(num_labels)) * 100
        p_y0 = 100 - p_y1
        
        b = (p_y1 < self.minSplitPercent) | (p_y0 < self.minSplitPercent)
        return a | b
    
    def terminate(self, node, y):
        #converts the node in question into a leaf by changing its 'leaf' status to true
        #it also assigns it a class based on the y data
        #Inputs:
        # node : The node that is a leaf
        # y    : The labels that will determine if the class is 1 or 0
        if (sum(y) / float(len(y))) >= 0.5:
            node['class'] = 1
            node['leaf'] = True
        else:
            node['class'] = 0
            node['leaf'] = True
            
    
    def learn(self, X, y):
        # call recursive_learn on the root node
        self.recursive_learn(X, y, self.tree);
        
    def recursive_learn(self, X, y, node):
        # Inputs:
        #   X    : data containing all attributes. This X only contains the data for the current node.
        #   y    : labels of datapoints for the current node.
        #   node : current node we're working with. A dictionary that has keys such as 'depth'. Example 
        #          node structures that you can use are:
        #          - For a leaf at depth 2 that outputs class 1: {'depth' : 2, 'leaf' : True, 'class' : 1}  
        #          - For a non-leaf node at depth 4: {'depth' : 4, 'leaf' : False, 'left' : {}, 'right' : {}}
        #            where 'left' and 'right' keys will be the left and right children, which are themselves 
        #            dictionaries.
        #
        # This method decides whether the current node should be split further or set as a leaf (with the appropriate
        # class);
        #
        # Recommended code structure (not required though)
        # 1) Check if 'node' should be split. If not, then set it as leaf and set it's class.
        # 2) Otherwise, find the best attribute to split on.
        #    - Note that since the the categorical variables here have non-binary values (e.g. 0,1,2,...k)
        #      and given that we're working with a binary tree, you'll have to find the best attribute value
        #      to split on for each attribute. 
        # 3) Once the best attribute is found, create two descendent nodes (left and right), set their attributes 
        #    (e.g. depth), and split the data (X and y) based on the attribute and value selected above. 
        #    You should use the partition_classes function implemented earlier.
        # 4) Call the "recursive_learn" function on each of the descendents. E.g.
        #    self.recursive_learn(X_left,  y_left,  node["left"])
        #    self.recursive_learn(X_right, y_right, node["right"])
        #
        # TODO: Your code here.
        # Note: This method does not return anything. It just builds the tree through self.tree
        X = np.array(X)
        if self.shouldStopSplitting(y , node['depth']):
            self.terminate(node, y)
        else: 
            node['leaf'] = False
            dims = np.shape(X)
            infogain_curr = 0
            final_splitVal = 0 
            final_idx = 0
            
            for i in range(0, dims[1]):
                feature_n = X[:, i]
                unique_f = np.unique(feature_n)
                for splitVal in unique_f:
                    splitClass = partition_classes(X,y,i,splitVal)
                    ig = information_gain(y,[splitClass[1], splitClass[3]])
                    if ig >= infogain_curr:
                        infogain_curr = ig
                        final_splitVal = splitVal
                        final_idx = i
            final_split = partition_classes(X, y, final_idx, final_splitVal)
            left = {'depth' : node['depth'] + 1}
            right = {'depth' : node['depth'] + 1}
            node['left'] = left
            node['right'] = right
            node['splitVal'] = final_splitVal
            node['split_idx'] = final_idx
            self.recursive_learn(final_split[0], final_split[1], node['left'])
            self.recursive_learn(final_split[2], final_split[3], node['right'])
            
    def classify(self, record):
        # Inputs:
        #   record : a new, single x data point 
        #
        # TODO: using the tree (self.tree), determine and return the class (0 or 1) of this data point.
        return self.rec_classify(record, self.tree)
    
    def rec_classify(self, record, node):
        
        if node['leaf']:
            return node['class']
        else:
            if record[node['split_idx']] == node['splitVal']:
                return self.rec_classify(record, node['left'])
            else:
                return self.rec_classify(record, node['right'])
                
        


# Testing your code

Below is a function that tests your decision tree by loading a Mushroom dataset (edible vs poisonous) and computes the training accuracy of the decision tree. If everything above is done correctly, you should get an accuracy of 0.9852

In [11]:
def main():
    X = list()
    y = list()
    numerical_cols = set([]) # indices of numeric attributes (columns)

    # Loading data set
    print 'reading data'
    with open("agaricus-lepiota.csv") as f:
        next(f, None)

        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                xline.append(line[i])

            X.append(xline[:-1])
            y.append(int(xline[-1]))
    
    d = DecisionTree();
    d.learn(X, y);
    
    y_predicted = [d.classify(record) for record in X];

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print "training accuracy: %.4f" % accuracy
    

if __name__ == "__main__":
    main()

reading data
training accuracy: 0.9852
